# Harry's House

Il nostro obiettivo e' di raccogliere informazioni sull'album in uscita il 20 Maggio 2022. Le domande a cui cercheremo di rispondere sono le sequenti:
1. _Quali sono i paesi in cui si è più discusso dell’album? Corrispondono ai paesi in cui l’album o le canzoni sono in vetta alle classifiche? => **Analisi delle publics metrics confrontate con i dati di Spotify**_ 
2. _Durante il giorno dell’uscita dell’album, l’hashtag #harryhouse è stato in tendenza? Anche nei giorni precedenti e successivi? Se si per quanto tempo?_
3. _Le canzoni più citate hanno per caso impattato altri social? Sono diventati brani in tendenza su altri social (tiktok)? => **Analisi della distribuzione degli hashtag e del trand**_
4. _Dalle reti delle menzioni di Twitter è possibile estrarre un buon modo per stilare una classifica simile se non uguale a quella di Spotify? ⇒ **Review sugli indici di centralità**_
5. _Quale è l’andamento delle varie canzoni dopo l’uscita dell’album? Il trend delle citazioni è simile al trend delle canzoni su Spotify (classifica globale)?_
6. _Quale è il sentimento percepito dai follower prima dell’uscita, il giorno dell’uscita e dopo?  ⇒ **Sentiment Analysis**_
7. _Quale è la tipologia di utenza? E’ cambiata durante i giorni antecedenti e post all’uscita dell’album? => **Analisi delle community**_
8. _è possibile identificare community di utenti, outliers, che si sono aggiunti alle discussioni? ⇒ **User Clustering**_

## Download dei dati necessari

Iniziamo a richiedere i dati necessari alla nostra analisi, i dati raccolti comprendono un arco di tempo che va dal 10 Maggio 2022 fino al (?)

In [2]:
#IMPORT

import tweepy
import CONFIG
import csv
import os
import json
import numpy as np
import pandas as pd

from collections import Counter

In [3]:
path = os.getcwd() + '\HARRY_HOUSE_DATA'
os.chdir(path)
print(os.getcwd())

C:\Users\sarac\OneDrive - Università degli Studi di Milano\UNI 2022\_SOCIALMEDIA\#harryhouse\HARRY_HOUSE_DATA


In [3]:
client = tweepy.Client(bearer_token = CONFIG.BEARER_TOKEN)

In [4]:
query_harryshouse = "HarrysHouse -is:retweet"

### Chiamata per prendere gli id dei tweet

In [5]:
list_tweet_id = []

for t in tweepy.Paginator(
    client.search_recent_tweets,
    query = query_harryshouse,
    max_results = 100,
    
).flatten():
    list_tweet_id.append(t.id)

print(len(list_tweet_id))



9324


In [6]:
json.dump(list_tweet_id, open('tweets_id_1705.json', 'w'))

In [4]:
list_tweet = json.load(open('tweets_id_1705.json'))

### Estensione dei campi dei tweet

In [5]:
n = len(list_tweet)/100
max_richieste = int(np.ceil(n))
max_richieste

94

In [6]:
tweets_extended = []
count = 0

for i in range(max_richieste):
    count += 1

    min_valore = min((i+1)*100, len(list_tweet))
    sub_list = list_tweet[(i*100) : min_valore]
    
    response = client.get_tweets(
        ids = sub_list,
        tweet_fields = [
            'created_at', 
            'lang', 
            'entities', 
            'geo', 
            'public_metrics'],
        expansions = ['author_id', 'geo.place_id'],
        user_fields = ['location', 
                       'verified', 
                       'public_metrics', 
                       'id', 
                       'username',
                       'name'
                      ]
    )
    
    users = {u.data['id']: u.data for u in response.includes['users']}

    for t in response.data:
        tweet_dict = t.data
        tweet_dict['author'] = users[tweet_dict['author_id']]
        tweets_extended.append(tweet_dict)
    
        #INFO TWEET
    
        #print("tweet id - ", t.id)
        print("lang - ", t.lang)
        print("date creation - ", t.created_at)
        print("entities: ", t.entities)
        print("public metrics: ", t.public_metrics)
        if t.geo:
            print("geo: ", t.geo)
    
    
        #INFO USER

        if users[t.data['author_id']]:
            id_author = t.data['author_id']
            author = users[id_author]
            
            
            print("username author: ", author['username'])
            print("verified: ", author['verified'])


            print("public_metrics")
            author = user['public_metrics']
            print("- follower count: ", u['followers_count'])
            print("- following count: ", u['following_count'])
            print("- tweet count: ", u['tweet_count'])
            
            try:
                location = author['location']
            except:
                location = ""
             
            print("location: " + location)

        #INFO PLACES
        
        try:
            place = places[t.geo['place_id']]
        except:
            place = ""
            
        print("place: ", place)
        if t.geo['place_id']:
            place = places[t.geo['place_id']]
            print(place.full_name)
    
   
        print("---------")
        print()
        

NameError: name 'client' is not defined

In [11]:
#FIXME

json.dump(tweets_extended, open('tweets_extended_1705.json', 'w'))

In [7]:
#FIXME

tweets_extended = json.load(open('tweets_extended_1705.json'))

## Analisi del contenuto ottenuto

### Tweets

#### Hashtag

In [4]:
hashtags = []

for t in tweets_extended:
    try:
        tweets = t['entities'].get('hashtags')
        for x in tweets:
            hashtags.append(x['tag'])
    except:
        continue


In [5]:
print(len(hashtags))

17753


Andiamo a contare i dati prodotti

In [6]:
c_hashtags = Counter(hashtags)

pd.DataFrame(
    c_hashtags.most_common()[:20],
    columns = ['Hashtag', 'Tot']
)

,Hashtag,Tot
0,HarrysHouse,8919
1,HarryStyles,2092
2,AsItWas,414
3,harrystyles,405
4,harryshouse,301
5,HarryStylesAvecNRJ,207
6,harrystylesONO,172
7,ONOTICKETS,158
8,OneNightOnly,123
9,YouAreHome,114


Esportiamo tutto in un file csv

In [16]:
head = ["Hashtag", "Tot"]
rows = []
for v in  c_hashtags.most_common()[:20]:
    row = []
    row.append(v[0])
    row.append(str(v[1]))
    rows.append(row)
    
with open('tot_hashtags_1705.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(head)
    writer.writerows(rows)
    

Esportiamo tutto su un file json

In [25]:
hasthags_to_save = {}

for c in c_hashtags:
    if c_hashtags[c] > 1:
        hasthags_to_save[c] = c_hashtags[c]

print(hasthags_to_save)

{'HarrysHouse': 8919, 'HarryStyles': 2092, 'harrystylesono': 41, 'onenightonly': 14, 'ONOTICKETS': 158, 'harrystyles': 405, 'AsItWas': 414, 'TodayShow': 6, 'fanpasses': 2, 'NYC': 4, 'hshq': 13, 'LoveOnTour2022': 87, 'onoharrystyles': 48, 'RegístrateGratis': 3, 'YouAreHome': 114, 'HARRYCHELLA': 27, 'Harrystylesconcert': 16, 'HarryStylesLoveOnTour2022': 19, 'onoroff': 2, 'ONO': 56, 'ENGLAND': 2, 'listeningparty': 16, 'OneNightOnly': 123, 'harry': 33, 'harrystylesONO': 172, 'harrystylestickets': 110, 'onenightonlytickets': 5, 'harrytickets': 7, 'AmberHeard': 2, 'cryptocurrency': 5, 'DeppVsHeard': 11, 'ElonMusk': 2, 'GMT': 2, 'HarshadChopda': 2, 'JohnnyDepp': 4, 'LTWTSantiago': 3, 'MoveToEarn': 2, 'fineline': 22, 'harryshouse': 301, 'loveontour': 55, 'hslot': 24, 'Coachella': 10, 'ONOlondon': 3, 'loveontourparis': 2, 'DontWorryDarling': 12, 'mypoliceman': 8, 'FreeHarry': 12, 'TreatPeopleWithKindness': 5, 'ChooseLove': 9, 'ono': 64, 'NY': 6, 'harries': 25, 'HarryStylesOneNightOnly': 6, 'tic

In [26]:
json.dump(hasthags_to_save, open('hashtags_1705.json', 'w'))

#### Mentions

In [15]:
mentions = []

for t in tweets_extended:
    try:
        tweets = t['entities'].get('mentions')
        for x in tweets:
            mentions.append(x['username'])
    except:
        continue


In [16]:
print(len(mentions))

3115


Andiamo a contare le menzioni

In [17]:
c_mentions = Counter(mentions)

pd.DataFrame(
    c_mentions.most_common()[:20],
    columns = ['Mentions', 'Tot']
)

,Mentions,Tot
0,Harry_Styles,661
1,Z100NewYork,243
2,HSHQ,188
3,NRJhitmusiconly,128
4,sonymusicph,74
5,zanelowe,68
6,youarehome,49
7,sottluvr,45
8,sweetapoIIo,42
9,adorefiIm,36


Esportiamo tutto su un file csv

In [55]:
head = ["Mentions", "Tot"]
rows = []
for v in c_mentions.most_common()[:20]:
    row = []
    row.append(v[0])
    row.append(str(v[1]))
    rows.append(row)
    
with open('tot_mentions_1705.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(head)
    writer.writerows(rows)
  

### Users

Andiamo a stampare le informazioni sugli utenti

In [13]:
list_users = {}

for t in tweets_extended:

    u = t['author']
    
    id_user = u['id']
    username = u['username']
    isVerified = u['verified']
        
    user = u['public_metrics']
    followers = user['followers_count']
    following = user['following_count']
    tweet_count =  user['tweet_count']
    
    try:
        location = u['location']
    except:
        location = ""
        
    list_users[id_user] = [
        id_user,
        username,
        isVerified,
        followers,
        following,
        tweet_count,
        location.replace("\n", " ")
    ]


In [9]:
len(list_users)

4952

In [11]:
json.dump(list_users , open('users_1705.json', 'w'))

In [60]:
head = ["Id User", "Username", "Verified", "Followers", "Following", "Tweet Count", "Location"]

with open('users_1705.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(head)
    writer.writerows(list_users.values())    